# Initialization

##### Imports

In [1]:
import os, shutil, importlib
import utils; importlib.reload(utils)

# os.environ["CUDA_VISIBLE_DEVICES"]=''
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import h5py
import keras
import sklearn

from glob import glob
from PIL import Image
from os.path import join as jp

from keras import models, optimizers, regularizers, applications
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.layers import *
from keras.optimizers import Adam

from scipy import ndimage

from sklearn import metrics
from sklearn.model_selection import train_test_split

from IPython.display import FileLink

%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

Using TensorFlow backend.


##### Struct Dir

In [3]:
path = '../data/fish/'
classes=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
utils.create_dirs(path, classes)

Move imgs into valid and copy 100 per class into sample

In [22]:
for cl in classes:
    imgs = glob(jp(path, 'train', cl, '*.jpg'))
    imgs_train, imgs_valid = train_test_split(imgs, test_size=0.2)
    for img in imgs_valid:
        shutil.move(img, jp(path, 'valid', cl))

Copy 100 of each class into sample

In [25]:
for cl in classes:
    take = 100 if len(glob(jp(path, 'valid', cl, '*.jpg'))) > 100 \
                else len(glob(jp(path, 'valid', cl, '*.jpg')))
    imgs_train = np.random.choice(glob(jp(path, 'train', cl, '*.jpg')), take, replace=False)
    imgs_valid = np.random.choice(glob(jp(path, 'valid', cl, '*.jpg')), take, replace=False)
    for img_tr, img_v in zip(imgs_train, imgs_valid):
        shutil.copy(img_tr, jp(path, 'sample', 'train', cl, img_tr.split('/')[-1]))
        shutil.copy(img_v, jp(path, 'sample', 'valid', cl, img_v.split('/')[-1]))

In [26]:
files = np.random.choice(glob(jp(path, 'test', 'test', '*.jpg')), 100, replace=False)
for img in files:
    shutil.copy(img, jp(path, 'sample', 'test', 'test', img.split('/')[-1]))

# Sample

## Define Paths

In [ ]:
path = '../data/statefarm/'
# path = '../data/statefarm/sample/'
path_model = '../data/statefarm/models/'
path_test = path + 'test'
path_train = path + 'train'
path_valid = path + 'valid'
path_img_arrays = path + 'img_arrays/'

path_submit = path_model + 'submissions/'
path_checkpoint = path_model + 'checkpoints/'
path_results = path_model + 'results/'